In [5]:
from cycling_manager.preprocess import get_data, preprocess, split
from cycling_manager.new_sequences import get_num_sequence, get_images, get_full_sequence

In [6]:
df = preprocess(get_data())

full df shape (603619, 25)
full df shape (603619, 25)
Score vertical meters - model: DecisionTreeRegressor() score: 0.9995649935680297
Score vertical meters - model: DecisionTreeRegressor() score: 0.9999393254142834
Score vertical meters - model: KNeighborsClassifier() score: 0.9946432086869127


In [7]:
train, test = split(df, 2021, 2022)
train.head()

,name,year,race_name
0,tadej-pogacar,2021,tour-de-france
369,sepp-kuss,2021,vuelta-a-espana
363,mikel-landa,2021,giro-d-italia
364,krists-neilands,2021,giro-d-italia
365,rafal-majka,2021,vuelta-a-espana


In [8]:
X_decoder_num_ls, y_decoder_ls, X_encoder_num_no_y_ls, y_encoder_num_ls, X_img_decoder_ls, X_img_encoder_ls, y_img_encoder_ls =\
    get_full_sequence(df, train, maxlen_num=40, maxlen_img=20, img=True, binary=True, y_encoder = True, num_cap=9999, img_cap=20, resnet=False)

[ WARN:0@24.582] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@24.582] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@24.887] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@25.027] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021

(519, 21, 5) (519, 21, 1) (519, 40, 14) (519, 40, 1) (519, 21, 150, 300, 3) (519, 20, 150, 300, 3) (519, 20)


In [9]:
X_decoder_num_test, y_decoder_test, X_encoder_num_no_y_test, y_encoder_num_test, X_img_decoder_test, X_img_encoder_test, y_img_encoder_test =\
    get_full_sequence(df, train, maxlen_num=40, maxlen_img=20, img=True, binary=True, y_encoder = True, num_cap=20, img_cap=20, resnet=False)

[ WARN:0@66.696] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-2.jpg'): can't open/read file: check file path/integrity
[ WARN:0@66.697] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021_stage-3.jpg'): can't open/read file: check file path/integrity
[ WARN:0@66.994] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2020_stage-4.jpg'): can't open/read file: check file path/integrity
[ WARN:0@67.138] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('../raw_data/img_300/czech-cycling-tour_2021

(519, 21, 5) (519, 21, 1) (519, 40, 14) (519, 40, 1) (519, 21, 150, 300, 3) (519, 20, 150, 300, 3) (519, 20)


In [10]:
from cycling_manager.model_comb import *

In [11]:
from tensorflow.keras.layers import Normalization

def numerical_model(num_encoder_features):

    masking = Masking(mask_value=-1000.)(num_encoder_features)
    
    normalizer = LayerNormalization()(masking)
    lstm1 = LSTM(units=20, dropout=0.2, return_sequences=True, activation='tanh')(normalizer)
    lstm2 = LSTM(units=40, dropout=0.2, return_sequences=True, activation='tanh')(lstm1)
    #output one time step from the sequence for each time step in the input but process 5 outputs of the input sequence at a time
    #y = TimeDistributed(Dense(units=5, activation='tanh'))(y)
    #attention_layer = attention()(y)
    lstm3 = LSTM(units=10, dropout=0.2, return_sequences=True, activation='tanh')(lstm2)
    dense1 = Dense(units=30, activation='relu')(lstm3)
    dense2 = Dense(units=10, activation='relu')(dense1)
    y = Dense(units=1, activation='linear')(dense2)
    
    return y
    
    

In [12]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

encoder_features = Input(shape=X_encoder_num_no_y_ls.shape[1:])

num_encoder_outputs = numerical_model(encoder_features)

print(num_encoder_outputs.shape)
print(encoder_features.shape)

num = Model(encoder_features, num_encoder_outputs)

num.compile(loss='mse', optimizer=Adam(), metrics='mae')

(None, 40, 1)
(None, 40, 14)


In [13]:

from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers.schedules import ExponentialDecay

initial_learning_rate = 0.001

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.05)

adam = Adam(learning_rate=lr_schedule)

num.compile(loss='mae', optimizer=adam, metrics='mae')

In [ ]:
es = EarlyStopping(monitor="val_loss",
                       patience=3,
                       restore_best_weights=True,
                       verbose=0)


num.fit(X_encoder_num_no_y_ls, y_encoder_num_ls, batch_size=16, epochs=1000, validation_split=0.2)

Epoch 1/1000
26/26 [==============================] - 10s 102ms/step - loss: 33.3752 - mae: 36.8493 - val_loss: 34.5226 - val_mae: 39.0361
Epoch 2/1000
26/26 [==============================] - 1s 24ms/step - loss: 33.2647 - mae: 36.7273 - val_loss: 34.5188 - val_mae: 39.0318
Epoch 3/1000
26/26 [==============================] - 1s 24ms/step - loss: 33.2961 - mae: 36.7619 - val_loss: 34.5194 - val_mae: 39.0326
Epoch 4/1000
26/26 [==============================] - 1s 24ms/step - loss: 33.2532 - mae: 36.7146 - val_loss: 34.5105 - val_mae: 39.0225
Epoch 5/1000
26/26 [==============================] - 1s 23ms/step - loss: 33.2641 - mae: 36.7266 - val_loss: 34.5099 - val_mae: 39.0218
Epoch 6/1000
26/26 [==============================] - 1s 24ms/step - loss: 33.2648 - mae: 36.7273 - val_loss: 34.5095 - val_mae: 39.0213
Epoch 7/1000
26/26 [==============================] - 1s 23ms/step - loss: 33.2638 - mae: 36.7262 - val_loss: 34.5143 - val_mae: 39.0267
Epoch 8/1000
26/26 [===================

KeyboardInterrupt: 

## Images

In [45]:
from tensorflow.keras.applications.resnet50 import ResNet50

def resnet_model(images, train_resnet=False):    
    # $CHALLENGIFY_BEGIN
    
    model = ResNet50(weights="imagenet", include_top=False, input_shape=images.shape[2:])
    
    if train_resnet == False:
        model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model  

def image_model(images, output=True, resnet=True, train_resnet=False):
    
    if not resnet:
        x = TimeDistributed(Rescaling(1./255, input_shape=(150,300,3)))(images)
        x = TimeDistributed(Conv2D(16, kernel_size=10, activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(3))(x)
        x = TimeDistributed(Conv2D(32, kernel_size=10, activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(3))(x)
        x = TimeDistributed(Conv2D(16, kernel_size=8, activation='relu'))(x)
        x = TimeDistributed(MaxPooling2D(3))(x)
    
    else:
        resnet = resnet_model(images, train_resnet=train_resnet)
        x = TimeDistributed(resnet)(images)
        x = TimeDistributed(Flatten())(x)
        
    x = TimeDistributed(Dense(200, activation='tanh'))(x)
    x = TimeDistributed(Dense(30, activation='tanh'))(x)
    y = TimeDistributed(Dense(10, activation='tanh'))(x)
    y = TimeDistributed(Dense(1, activation='linear'))(y)
    
    if output:
        print(y)
        return y
    else:
        return x

In [46]:
list(X_img_encoder_ls.shape[2:])

[150, 300, 3]

In [47]:
img_features = Input(shape=tuple(X_img_encoder_ls.shape[1:]))

img_encoder_outputs = image_model(img_features)

print(img_features.shape)
print(img_encoder_outputs.shape)

img = Model(img_features, img_encoder_outputs)

img.compile(loss='mse', optimizer=Adam(), metrics='mae')

KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1), dtype=tf.float32, name=None), name='time_distributed_47/Reshape_1:0', description="created by layer 'time_distributed_47'")
(None, 20, 150, 300, 3)
(None, 20, 1)


In [48]:

initial_learning_rate = 0.001

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=1000, decay_rate=0.05)

adam = Adam(learning_rate=lr_schedule)

img.compile(loss='mae', optimizer=adam, metrics='mae')

In [49]:
img.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 20, 150, 300, 3)  0         
                             ]                                   
                                                                 
 time_distributed_42 (TimeDi  (None, 20, 5, 10, 2048)  23587712  
 stributed)                                                      
                                                                 
 time_distributed_43 (TimeDi  (None, 20, 102400)       0         
 stributed)                                                      
                                                                 
 time_distributed_44 (TimeDi  (None, 20, 200)          20480200  
 stributed)                                                      
                                                                 
 time_distributed_45 (TimeDi  (None, 20, 30)           6030

In [50]:
es = EarlyStopping(monitor="val_loss",
                       patience=3,
                       restore_best_weights=True,
                       verbose=0)


img.fit(X_img_encoder_ls, y_img_encoder_ls, batch_size=8, epochs=1000, validation_split=0.2)

Epoch 1/1000
24/52 [============>.................] - ETA: 1:26 - loss: 5.1597 - mae: 5.1597